# Método Simplex Tableau
### Projeto M210 – Otimização

Preencha os coeficientes abaixo (até 4 variáveis e 4 restrições)
e execute todas as células para obter o resultado.

In [ ]:
# Quantidade de variáveis e restrições do exemplo
n_vars = 4   # número de variáveis de decisão (x1, x2, x3, x4)
n_rest = 4   # número de restrições

# Coeficientes da função objetivo (Max Z)
# Exemplo: Z = 3x1 + 5x2 + 2x3 + 4x4
c = [3, 5, 2, 4]

# Matriz das restrições (lado esquerdo A·x)
# Cada linha representa uma restrição
A = [
    [2, 1, 1, 0],   # 2x1 + 1x2 + 1x3 + 0x4 <= 8
    [1, 2, 0, 1],   # 1x1 + 2x2 + 0x3 + 1x4 <= 8
    [1, 1, 2, 3],   # 1x1 + 1x2 + 2x3 + 3x4 <= 10
    [0, 2, 1, 1]    # 0x1 + 2x2 + 1x3 + 1x4 <= 6
]

# Lado direito das restrições (vetor b)
b = [8, 8, 10, 6]


In [ ]:
import numpy as np
from prettytable import PrettyTable

def simplex_tableau(c, A, b):
    """Monta o tableau inicial do método Simplex (forma padrão Max)."""
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    m, n = A.shape  # m = nº restrições, n = nº variáveis de decisão

    # Tamanho do tableau: m linhas de restrições + 1 linha de Z
    # Colunas: n variáveis + m folgas + 1 termo independente (b)
    tableau = np.zeros((m + 1, n + m + 1), dtype=float)

    # Parte das variáveis de decisão
    tableau[:m, :n] = A
    # Identidade para variáveis de folga
    tableau[:m, n:n + m] = np.eye(m)
    # Coluna b (lado direito)
    tableau[:m, -1] = b
    # Linha da função objetivo (coeficientes negativos para Max)
    tableau[-1, :n] = -np.array(c, dtype=float)
    return tableau


def mostrar_tableau(tableau):
    """Mostra o tableau com nomes de linhas e colunas para ficar mais legível."""
    m = tableau.shape[0] - 1          # nº restrições
    total_cols = tableau.shape[1]
    n = total_cols - m - 1            # nº variáveis de decisão

    # Nomes das colunas: x1..xn, s1..sm, b
    colunas = [f"x{i+1}" for i in range(n)] + [f"s{i+1}" for i in range(m)] + ["b"]

    t = PrettyTable()
    t.field_names = ["Linha"] + colunas

    for idx, row in enumerate(tableau):
        if idx < m:
            nome_linha = f"R{idx+1}"
        else:
            nome_linha = "Z"
        t.add_row([nome_linha] + [f"{val:.2f}" for val in row])
    print(t)


def simplex(c, A, b):
    """Executa o método Simplex (tableau) e mostra as iterações de forma organizada."""
    tableau = simplex_tableau(c, A, b)
    m, n = len(A), len(A[0])
    iteracao = 0

    print("Tableau inicial:")
    mostrar_tableau(tableau)

    # Enquanto houver coeficiente negativo na linha de Z (colunas das variáveis)
    while any(tableau[-1, :-1] < 0):
        iteracao += 1
        print("\n" + "=" * 70)
        print(f"Iteração {iteracao}:")

        # Escolha da coluna pivô (variável que entra na base)
        col_pivo = int(np.argmin(tableau[-1, :-1]))

        # Verifica se o problema é ilimitado
        if all(tableau[:-1, col_pivo] <= 0):
            print("Problema ilimitado (sem solução ótima finita).")
            return None, None

        # Razão mínima para escolher a linha pivô (variável que sai da base)
        razoes = tableau[:-1, -1] / tableau[:-1, col_pivo]
        razoes[tableau[:-1, col_pivo] <= 0] = np.inf
        lin_pivo = int(np.argmin(razoes))

        print(f"Coluna pivô: {col_pivo}  |  Linha pivô: {lin_pivo}")

        # Normaliza a linha pivô
        pivo = tableau[lin_pivo, col_pivo]
        tableau[lin_pivo, :] /= pivo

        # Zera as demais posições da coluna pivô
        for i in range(tableau.shape[0]):
            if i != lin_pivo:
                fator = tableau[i, col_pivo]
                tableau[i, :] -= fator * tableau[lin_pivo, :]

        print("Tableau após o pivoteamento:")
        mostrar_tableau(tableau)

    print("\n" + "=" * 70)
    print("Solução ótima encontrada:")
    mostrar_tableau(tableau)

    # Recupera os valores ótimos das variáveis de decisão
    n_vars = len(c)
    x = np.zeros(n_vars)
    for j in range(n_vars):
        col = tableau[:-1, j]
        # Verifica se é coluna básica (um 1 e o resto 0)
        if np.isclose(col, 0).sum() == (len(col) - 1) and np.isclose(col, 1).sum() == 1:
            lin = int(np.where(np.isclose(col, 1))[0][0])
            x[j] = tableau[lin, -1]

    z_otimo = tableau[-1, -1]

    print("\nValores ótimos das variáveis:")
    for i, val in enumerate(x, start=1):
        print(f"x{i} = {val:.4f}")
    print(f"\nValor ótimo da função objetivo Z* = {z_otimo:.4f}")

    return x, z_otimo


# Executa o método Simplex para os dados definidos na célula anterior
x, z = simplex(c, A, b)
